# Training MMAction3 Mask-RCNN Model on Sagemaker Distributed Cluster

## Motivation
[MMDetection](https://github.com/open-mmlab/mmdetection) is a popular open-source Deep Learning framework focused on Computer Vision models and use cases. MMDetection provides to higher level APIs for model training and inference. It demonstrates [state-of-the-art benchmarks](https://github.com/open-mmlab/mmdetection#benchmark-and-model-zoo) for variety of model architecture and extensive Model Zoo.

In this notebook, we will build a custom training container with MMdetection library and then train Mask-RCNN model from scratch on [COCO2017 dataset](https://cocodataset.org/#home) using Sagemaker distributed [training feature](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-training.html) in order to reduce training time.

### Preconditions
- To execute this notebook, you will need to have COCO 2017 training and validation datasets uploaded to S3 bucket available for Amazon Sagemaker service.


## Building Training Container

Amazon Sagemaker allows to BYO containers for training, data processing, and inference. In our case, we need to build custom training container which will be pushed to your AWS account [ECR service](https://aws.amazon.com/ecr/). 

For this, we need to login to public ECR with Sagemaker base images and private ECR reposity.

In [4]:
import sagemaker, boto3

session = sagemaker.Session()
region = session.boto_region_name
account = boto3.client('sts').get_caller_identity().get('Account')
bucket = session.default_bucket()

In [5]:
# login to Sagemaker ECR with Deep Learning Containers
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin 763104351884.dkr.ecr.{region}.amazonaws.com
# login to your private ECR
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account}.dkr.ecr.{region}.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


Now, let review training container:
- use Sagemaker PyTorch 1.5.0 container as base image;
- install latest version of Pytorch libraries and MMdetection dependencies;
- build MMDetection from sources;
- configure Sagemaker env variables, specifically, what script to use at training time.

In [ ]:
! pygmentize -l docker Dockerfile.training

<br>
<br>
Next, we build and push custom training container to private ECR
<br>
<br>

In [7]:
! ./build_and_push.sh mmaction2-training latest Dockerfile.training

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  36.86MB
Step 1/13 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.5.0-gpu-py36-cu101-ubuntu16.04
 ---> 47cd15520b75
Step 2/13 : LABEL author="vadimd@amazon.com"
 ---> Using cache
 ---> 78da0851d3c4
Step 3/13 : WORKDIR /opt/ml/code
 ---> Using cache
 ---> 07bd9a0de06d
Step 4/13 : RUN pip install --upgrade --force-reinstall  torch torchvision cython
 ---> Using cache
 ---> b13c99508ae7
Step 5/13 : RUN pip install mmcv-full==latest+torch1.7.0+cu101 -f https://download.openmmlab.com/mmcv/dist/index.html
 ---> Using cache
 ---> 7e525985aced
Step 6/13 : RUN git clone https://github.com/open-mmlab/mmaction2.git
 ---> Usin

### Training script

At training time, Sagemaker executes training script defined in `SAGEMAKER_PROGRAM` variable. In our case, this script does following
- parses user parameters passed via Sagemaker Hyperparameter dictionary;
- based on parameters constructs launch command;
- uses `torch.distributed.launch` utility to launch distributed training;
- uses MMDetection `tools/train.py` to configure trianing process.


In [ ]:
! pygmentize container_training/mmaction2_train.py

## Start Sagemaker Training 

In [8]:
# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [9]:
from time import gmtime, strftime

prefix_input = 'mmaction2-input'
prefix_output = 'mmaction2-ouput'

In [10]:
container = "mmaction2-training" # your container name
tag = "latest"
image = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account, region, container, tag)

In [14]:
# algorithm parameters

hyperparameters = {
    "config-file" : "configs/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics400_rgb.py", # config path is relative to MMDetection root directory
    "dataset" : "kinetics400_tiny",
    "auto-scale" : "false", # whether to scale LR and Warm Up time
    "validate" : "true", # whether to run validation after training is done
    
    # 'options' allows to override individual config values
    "options" : "total_epochs=1; optimizer.lr=0.08; evaluation.gpu_collect=True",
}

In [15]:
# Sagemaker will parse metrics from STDOUT and store/visualize them as part of training job
metrics = [
    {
        "Name": "top_k_accuracy",
        "Regex": ".*top_k_accuracy:\s([0-9\\.]+)\s*"
    },
    {
        "Name": "mean_class_accuracy",
        "Regex": ".*mean_class_accuracy:\s([0-9\\.]+)\s*"
    },
]

<br>
<br>

Execute cell below to start training on Sagemaker.
<br>
<br>

In [9]:
# download, decompress the data
!rm kinetics400_tiny.zip*
!rm -rf kinetics400_tiny
!wget https://download.openmmlab.com/mmaction/kinetics400_tiny.zip
!unzip kinetics400_tiny.zip > /dev/null

rm: cannot remove ‘kinetics400_tiny.zip*’: No such file or directory
--2020-12-10 12:43:21--  https://download.openmmlab.com/mmaction/kinetics400_tiny.zip
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.35
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18308682 (17M) [application/zip]
Saving to: ‘kinetics400_tiny.zip’

kinetics400_tiny.zi 100%[===================>]  17.46M  8.76MB/s    in 2.0s    

2020-12-10 12:43:24 (8.76 MB/s) - ‘kinetics400_tiny.zip’ saved [18308682/18308682]



In [1]:
# Check the directory structure of the tiny data

# Install tree first
!sudo yum update -y && sudo yum install -y tree
!tree kinetics400_tiny

Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper,
              : versionlock
No packages marked for update
Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper,
              : versionlock
Package tree-1.6.0-5.8.amzn1.x86_64 already installed and latest version
Nothing to do
kinetics400_tiny
├── kinetics_tiny_train_video.txt
├── kinetics_tiny_val_video.txt
├── train
│   ├── 27_CSXByd3s.mp4
│   ├── 34XczvTaRiI.mp4
│   ├── A-wiliK50Zw.mp4
│   ├── D32_1gwq35E.mp4
│   ├── D92m0HsHjcQ.mp4
│   ├── DbX8mPslRXg.mp4
│   ├── FMlSTTpN3VY.mp4
│   ├── h10B9SVE-nk.mp4
│   ├── h2YqqUhnR34.mp4
│   ├── iRuyZSKhHRg.mp4
│   ├── IyfILH9lBRo.mp4
│   ├── kFC3KY2bOP8.mp4
│   ├── LvcFDgCAXQs.mp4
│   ├── O46YA8tI530.mp4
│   ├── oMrZaozOvdQ.mp4
│   ├── oXy-e_P_cAI.mp4
│   ├── P5M-hAts7MQ.mp4
│   ├── phDqGd0NKoo.mp4
│   ├── PnOe3GZRVX8.mp4
│   ├── R8HXQkdgKWA.mp4
│   ├── RqnKtCEoEcA.mp4
│   ├── soEcZZsBmDs.mp4
│   ├── TkkZPZHbAKA.mp4
│   ├── T_TMNGzVrDk.mp4
│ 

In [2]:
# After downloading the data, we need to check the annotation format
!cat kinetics400_tiny/kinetics_tiny_train_video.txt

D32_1gwq35E.mp4 0
iRuyZSKhHRg.mp4 1
oXy-e_P_cAI.mp4 0
34XczvTaRiI.mp4 1
h2YqqUhnR34.mp4 0
O46YA8tI530.mp4 0
kFC3KY2bOP8.mp4 1
WWP5HZJsg-o.mp4 1
phDqGd0NKoo.mp4 1
yLC9CtWU5ws.mp4 0
27_CSXByd3s.mp4 1
IyfILH9lBRo.mp4 1
T_TMNGzVrDk.mp4 1
TkkZPZHbAKA.mp4 0
PnOe3GZRVX8.mp4 1
soEcZZsBmDs.mp4 1
FMlSTTpN3VY.mp4 1
WaS0qwP46Us.mp4 0
A-wiliK50Zw.mp4 1
oMrZaozOvdQ.mp4 1
ZQV4U2KQ370.mp4 0
DbX8mPslRXg.mp4 1
h10B9SVE-nk.mp4 1
P5M-hAts7MQ.mp4 0
R8HXQkdgKWA.mp4 0
D92m0HsHjcQ.mp4 0
RqnKtCEoEcA.mp4 0
LvcFDgCAXQs.mp4 0
xGY2dP0YUjA.mp4 0
Wh_YPQdH1Zg.mp4 0


In [6]:
!aws s3 cp --recursive kinetics400_tiny s3://$bucket/kinetics400_tiny

upload: kinetics400_tiny/kinetics_tiny_train_video.txt to s3://sagemaker-us-east-1-579019700964/kinetics400_tiny/kinetics_tiny_train_video.txt
upload: kinetics400_tiny/kinetics_tiny_val_video.txt to s3://sagemaker-us-east-1-579019700964/kinetics400_tiny/kinetics_tiny_val_video.txt
upload: kinetics400_tiny/train/LvcFDgCAXQs.mp4 to s3://sagemaker-us-east-1-579019700964/kinetics400_tiny/train/LvcFDgCAXQs.mp4
upload: kinetics400_tiny/train/A-wiliK50Zw.mp4 to s3://sagemaker-us-east-1-579019700964/kinetics400_tiny/train/A-wiliK50Zw.mp4
upload: kinetics400_tiny/train/DbX8mPslRXg.mp4 to s3://sagemaker-us-east-1-579019700964/kinetics400_tiny/train/DbX8mPslRXg.mp4
upload: kinetics400_tiny/train/IyfILH9lBRo.mp4 to s3://sagemaker-us-east-1-579019700964/kinetics400_tiny/train/IyfILH9lBRo.mp4
upload: kinetics400_tiny/train/D92m0HsHjcQ.mp4 to s3://sagemaker-us-east-1-579019700964/kinetics400_tiny/train/D92m0HsHjcQ.mp4
upload: kinetics400_tiny/train/27_CSXByd3s.mp4 to s3://sagemaker-us-east-1-57901970

In [ ]:
est = sagemaker.estimator.Estimator(image,
                                          role=role,
                                          train_instance_count=2,
                                          train_instance_type='ml.p3.8xlarge',
                                          train_volume_size=100,
                                          output_path="s3://{}/{}".format(bucket, prefix_output),
                                          metric_definitions = metrics,
                                          hyperparameters = hyperparameters, 
                                          sagemaker_session=session
)

est.fit({"training" : "s3://"+bucket+"/kinetics400_tiny/"})

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2020-12-10 13:24:49 Starting - Starting the training job...
2020-12-10 13:25:14 Starting - Launching requested ML instancesProfilerReport-1607606688: InProgress
.........
2020-12-10 13:26:41 Starting - Preparing the instances for training...